In [1]:
import pandas as pd
pd.options.display.max_columns = 999
import pgeocode as geo
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

C:\Users\yjaey\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\yjaey\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True,

In [2]:
sc= pd.read_excel(r'Input\\social_capital.97-05-09.updated8.28.17.xlsx', sheet_name='2005')
sc.head(1)

,fips,areaname,relig05,civic05,bus05,pol05,prof05,labor05,bowl05,fitns05,golf05,sport05,pop05,pvote04,respn05,nccs05,assn05,sk05
0,1001,"Autauga, AL",40,4,3,1,0,5,1,3,2,0,49676,0.58,0.63,143.0,1.187696,-0.718073


In [3]:
cols_rename= {
            'areaname':'area',
            'fitns05': 'fit05'
            }

In [4]:
sc.describe()

,fips,relig05,civic05,bus05,pol05,prof05,labor05,bowl05,fitns05,golf05,sport05,pop05,pvote04,respn05,nccs05,assn05,sk05
count,3108.000000,3108.000000,3108.000000,3108.000000,3108.000000,3108.000000,3108.000000,3108.000000,3108.000000,3108.000000,3108.000000,3.108000e+03,3108.000000,3108.000000,3107.000000,3108.000000,3108.000000
mean,30685.834299,55.221364,10.141248,5.608108,0.954633,2.428893,5.153153,1.493565,10.238095,3.833333,0.247104,9.443713e+04,0.582578,0.642436,443.735114,1.431515,-0.000005
std,14982.915779,117.016467,24.845095,17.637835,4.447476,11.316657,16.022384,3.109568,31.043196,7.273800,1.052871,3.043623e+05,0.095764,0.088840,1339.935342,0.698885,1.392700
min,1001.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000e+01,0.200000,0.180000,2.000000,0.000000,-3.909425
25%,19046.500000,11.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.133975e+04,0.520000,0.590000,61.000000,0.989921,-0.936407
50%,29214.000000,23.000000,3.000000,2.000000,0.000000,0.000000,1.000000,1.000000,2.000000,2.000000,0.000000,2.546300e+04,0.580000,0.650000,127.000000,1.310621,-0.225902
75%,46009.500000,52.000000,9.000000,4.000000,0.000000,1.000000,3.000000,2.000000,7.000000,4.000000,0.000000,6.476950e+04,0.650000,0.710000,304.500000,1.708895,0.702159
max,56045.000000,3030.000000,625.000000,486.000000,149.000000,319.000000,322.000000,61.000000,743.000000,140.000000,23.000000,9.786373e+06,1.820000,0.850000,36697.000000,8.064516,14.296260


In [5]:
sc.rename(columns=cols_rename, inplace=True)
sc.columns= sc.columns.str.strip().str.lower()
sc.columns=sc.columns.str.replace('\d+','',regex=True).str.strip()
sc['nccs']= sc['nccs'].replace(np.nan, 0) # This is one of the key things that make results: Before this fix: corr= 0.54. After this fix corr= 0.99

sc['nccs']= sc['nccs']/(sc['pop']/1000)
sc['assn']= (sc['relig'] + sc['civic'] + sc['bus'] + sc['pol'] + sc['prof'] + sc['labor'] + sc['bowl'] + sc['fit'] + sc['golf'] + sc['sport']) / (sc['pop']/1000)

sc.drop(columns=['bowl', 'civic', 'fit', 'golf', 'relig', 'sport', 'pol', 'prof', 'bus', 'labor'], inplace=True)
sc.drop(columns=['pop'],inplace=True)
sc.head(3)

,fips,area,pvote,respn,nccs,assn,sk
0,1001,"Autauga, AL",0.58,0.63,2.878654,1.187696,-0.718073
1,1003,"Baldwin, AL",0.57,0.54,3.625534,1.276336,-0.868675
2,1005,"Barbour, AL",0.50,0.54,3.889107,0.927677,-1.486606


In [6]:
x = sc.loc[:, ['respn','pvote','assn','nccs']].values
# Separating out the target
y = sc.loc[:,['fips']].values
# Standardizing the features
x = StandardScaler().fit_transform(x)

pca = PCA(n_components=1)
principalComponents = pca.fit_transform(x)
pca_df = pd.DataFrame(data = principalComponents
             , columns = ['pc1'])
pca_df.head(2)

,pc1
0,-0.718186
1,-0.869019


In [7]:
sc['pc1']=pca_df[['pc1']]

In [8]:
sc.head()

,fips,area,pvote,respn,nccs,assn,sk,pc1
0,1001,"Autauga, AL",0.58,0.63,2.878654,1.187696,-0.718073,-0.718186
1,1003,"Baldwin, AL",0.57,0.54,3.625534,1.276336,-0.868675,-0.869019
2,1005,"Barbour, AL",0.50,0.54,3.889107,0.927677,-1.486606,-1.486880
3,1007,"Bibb, AL",0.47,0.54,2.585972,1.315670,-1.540171,-1.540479
4,1009,"Blount, AL",0.52,0.63,1.958846,0.878735,-1.431489,-1.431343


In [9]:
sc['sk'].corr(sc['pc1'])

0.9999999869969596

In [10]:
sc.shape

(3108, 8)

In [11]:
sc['year']=2005
sc.to_csv("output\\sc05.csv")